In [1]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
from empyrical import max_drawdown,sharpe_ratio,sortino_ratio
import warnings
warnings.filterwarnings("ignore") 

In [2]:
data = pd.read_csv("btcusdt_1h.csv")

In [3]:
def Heiken_Ashi(df):
    df['HA_Close']=(df['open']+ df['high']+ df['low']+ df['close'])/4
    df['HA_Open']=(df['open']+df['close'])/2   
    
    for i in range(1, len(df)):
        df['HA_Open'][i]=(df['HA_Open'][i-1]+df['HA_Close'][i-1])/2 
    df['HA_High']=df[['HA_Open','HA_Close','high']].max(axis=1)
    df['HA_Low']=df[['HA_Open','HA_Close','low']].min(axis=1)
    return

Heiken_Ashi(data)

In [2]:
# def ATR(df):
#     df['ATR'] = df['high'].combine(df['close'].shift(1), max) - df['low'].combine(df['close'].shift(1), min)
#     df['ATR'] = df['ATR'].ewm(span=11, adjust=False).mean()  
#     df['ATR_Signal'] = df['ATR'].ewm(span = 10, adjust = False).mean() #window can be changed
#     df['TP'] = (df['high'] + df['low'] + df['close']) / 3
#     period = 10
#     df['ATR_Signal_slope'] = (df['ATR_Signal'] - df['ATR_Signal'].shift(period)) / period
#     df['VWAP'] = (df['TP'] * df['volume']).cumsum() / df['volume'].cumsum()
#     return df

def ATR(df):
    df['TR'] = None
    for i in range(1, len(df)):
        high_low_range = abs(df['high'][i] - df['low'][i])
        high_prev_close_range = abs(df['high'][i] - df['close'][i - 1])
        low_prev_close_range = abs(df['low'][i] - df['close'][i - 1])
        df['TR'][i] = max(high_low_range, high_prev_close_range, low_prev_close_range)

    df['ATR'] = None
    df['ATR'] = df['TR'].ewm(span=11).mean()
    df['ATR_Signal'] = df['ATR'].ewm(span = 10, adjust = False).mean() #window can be changed
    period = 10
    df['ATR_Signal_slope'] = (df['ATR_Signal'] - df['ATR_Signal'].shift(period)) / period
    df['TP'] = (df['high'] + df['low'] + df['close']) / 3
    df['VWAP'] = (df['TP']).cumsum() * df['volume']/ df['volume'].cumsum()
    return df

In [5]:
data = ATR(data)

In [6]:
def macd(data):
    data['MACD'] =data['HA_Low'].ewm(span = 12,adjust=False).mean() + data['HA_Close'].ewm(span = 12,adjust=False).mean() + data['HA_Open'].ewm(span = 12,adjust=False).mean()+ data['HA_High'].ewm(span = 12,adjust=False).mean()- data['HA_Close'].ewm(span=26,adjust=False).mean()-data['HA_Open'].ewm(span=26,adjust=False).mean()-data['HA_High'].ewm(span = 26,adjust=False).mean()-data['HA_Low'].ewm(span = 12,adjust=False).mean()
    return data
data = macd(data)
data

,datetime,open,high,low,close,volume,HA_Close,HA_Open,HA_High,HA_Low,ATR,TP,VWAP,MACD,Signal_MACD
0,2018-01-01 05:30:00,13715.65,13715.65,13400.01,13529.01,443.356199,13590.0800,13622.330000,13715.6500,13400.010000,315.640000,13548.223333,13548.223333,0.000000,0.000000
1,2018-01-01 06:30:00,13528.99,13595.89,13155.38,13203.06,383.697006,13370.8300,13606.205000,13606.2050,13155.380000,336.451667,13318.110000,13441.466239,-18.610080,-3.722016
2,2018-01-01 07:30:00,13203.00,13418.43,13200.00,13330.18,429.064572,13287.9025,13488.517500,13488.5175,13200.000000,316.781389,13316.203333,13398.678949,-40.112555,-11.000124
3,2018-01-01 08:30:00,13330.26,13611.27,13290.00,13410.03,420.087030,13410.3900,13388.210000,13611.2700,13290.000000,317.529491,13437.100000,13408.307955,-46.229757,-18.046050
4,2018-01-01 09:30:00,13434.98,13623.29,13322.15,13601.01,340.807329,13495.3575,13399.300000,13623.2900,13322.150000,314.797909,13515.483333,13426.416995,-43.346282,-23.106097
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35203,2022-01-12 01:30:00,42972.04,43095.26,42692.19,42800.38,1219.601780,42889.9675,42669.867354,43095.2600,42669.867354,560.213227,42862.610000,20864.785538,252.310516,139.617491
35204,2022-01-12 02:30:00,42797.62,42823.69,42643.74,42659.20,702.103800,42731.0625,42779.917427,42823.6900,42643.740000,496.836023,42708.876667,20864.976576,269.040337,165.502061
35205,2022-01-12 03:30:00,42664.71,42776.14,42597.41,42713.13,561.859930,42687.8475,42755.489963,42776.1400,42597.410000,443.818352,42695.560000,20865.129359,275.697408,187.541130
35206,2022-01-12 04:30:00,42713.12,42886.28,42633.97,42729.29,681.142010,42740.6650,42721.668732,42886.2800,42633.970000,411.900293,42749.846667,20865.315035,282.499849,206.532874


In [7]:
data["MACD_STD"]= data['MACD'].rolling(window = 60).std()
data["UPPER_THRESHOLD"] = 1.5*data["MACD_STD"]
data["LOWER_THRESHOLD"] = -1.5*data["MACD_STD"]
data

,datetime,open,high,low,close,volume,HA_Close,HA_Open,HA_High,HA_Low,ATR,TP,VWAP,MACD,Signal_MACD,MACD_STD,UPPER_THRESHOLD,LOWER_THRESHOLD
0,2018-01-01 05:30:00,13715.65,13715.65,13400.01,13529.01,443.356199,13590.0800,13622.330000,13715.6500,13400.010000,315.640000,13548.223333,13548.223333,0.000000,0.000000,NaN,NaN,NaN
1,2018-01-01 06:30:00,13528.99,13595.89,13155.38,13203.06,383.697006,13370.8300,13606.205000,13606.2050,13155.380000,336.451667,13318.110000,13441.466239,-18.610080,-3.722016,NaN,NaN,NaN
2,2018-01-01 07:30:00,13203.00,13418.43,13200.00,13330.18,429.064572,13287.9025,13488.517500,13488.5175,13200.000000,316.781389,13316.203333,13398.678949,-40.112555,-11.000124,NaN,NaN,NaN
3,2018-01-01 08:30:00,13330.26,13611.27,13290.00,13410.03,420.087030,13410.3900,13388.210000,13611.2700,13290.000000,317.529491,13437.100000,13408.307955,-46.229757,-18.046050,NaN,NaN,NaN
4,2018-01-01 09:30:00,13434.98,13623.29,13322.15,13601.01,340.807329,13495.3575,13399.300000,13623.2900,13322.150000,314.797909,13515.483333,13426.416995,-43.346282,-23.106097,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35203,2022-01-12 01:30:00,42972.04,43095.26,42692.19,42800.38,1219.601780,42889.9675,42669.867354,43095.2600,42669.867354,560.213227,42862.610000,20864.785538,252.310516,139.617491,114.589102,171.883652,-171.883652
35204,2022-01-12 02:30:00,42797.62,42823.69,42643.74,42659.20,702.103800,42731.0625,42779.917427,42823.6900,42643.740000,496.836023,42708.876667,20864.976576,269.040337,165.502061,118.998774,178.498161,-178.498161
35205,2022-01-12 03:30:00,42664.71,42776.14,42597.41,42713.13,561.859930,42687.8475,42755.489963,42776.1400,42597.410000,443.818352,42695.560000,20865.129359,275.697408,187.541130,123.205699,184.808549,-184.808549
35206,2022-01-12 04:30:00,42713.12,42886.28,42633.97,42729.29,681.142010,42740.6650,42721.668732,42886.2800,42633.970000,411.900293,42749.846667,20865.315035,282.499849,206.532874,127.347871,191.021807,-191.021807


In [8]:
data['signals'] = 0
data['stoploss'] = 0
data["up"] = data["HA_Close"].rolling(window = 22).max()
data["down"] = data["HA_Close"].rolling(window = 21).min()
for i in range(60,len(data)):
    if(data["MACD"][i] >= data["UPPER_THRESHOLD"][i] and data["MACD"][i-1] < data["UPPER_THRESHOLD"][i-1] and data['ATR_Signal_slope'][i] > 0):
        data["signals"][i] = 1
        data["stoploss"][i] = data["up"][i] - 2.6 * data["ATR"][i]
    elif(data["MACD"][i] <= data["UPPER_THRESHOLD"][i] and data["MACD"][i-1] > data["LOWER_THRESHOLD"][i-1] and data['ATR_Signal_slope'][i] < 0):
        data['signals'][i] = -1
        data["stoploss"][i] = data["low"][i] - 2.5 * data["ATR"][i]
    

In [9]:
for i in range(len(data)):
    if data['signals'][i] == 1:
        sl_active = False
        sl = data['stoploss'][i]
        for j in range(i+1, len(data)):
            if data['signals'][j] == -1 or (data['HA_Close'][j] <= sl and sl_active):
                for k in range(i+1, j):
                    data['signals'][k] = 0
                break
            if data['HA_Close'][j] > sl:
                sl_active = True

    elif data['signals'][i] == -1:
        sl_active = False
        sl = data['stoploss'][i]
        for j in range(i+1, len(data)):
            if data['signals'][j] == 1 or (data['HA_Close'][j] >= sl and sl_active):
                for k in range(i+1, j):
                    data['signals'][k] = 0
                break
            if data['HA_Close'][j] < sl:
                sl_active = True
            


In [12]:
len(data[data['signals'] == 0])
# len(data)

34394

In [17]:
data[1800:1850]

,datetime,open,high,low,close,volume,HA_Close,HA_Open,HA_High,HA_Low,...,VWAP,MACD,Signal_MACD,MACD_STD,UPPER_THRESHOLD,LOWER_THRESHOLD,signals,stoploss,up,down
1800,2018-03-18 15:30:00,7692.00,7729.33,7532.09,7580.99,1974.603449,7633.6025,7702.095092,7729.330000,7532.090000,...,10199.548422,-133.294800,-138.737744,73.043938,109.565908,-109.565908,0,0.000000,8004.4675,7626.8525
1801,2018-03-18 16:30:00,7580.00,7810.97,7544.00,7760.09,1950.277686,7673.7650,7667.848796,7810.970000,7544.000000,...,10197.751316,-127.981740,-136.586543,73.820675,110.731013,-110.731013,0,0.000000,7988.5325,7626.8525
1802,2018-03-18 17:30:00,7761.19,7829.00,7669.82,7701.00,1652.698359,7740.2525,7670.806898,7829.000000,7669.820000,...,10196.247682,-116.794723,-132.628179,74.373042,111.559563,-111.559563,0,0.000000,7960.1850,7626.8525
1803,2018-03-18 18:30:00,7701.00,7728.51,7534.90,7606.00,2236.378527,7642.6025,7705.529699,7728.510000,7534.900000,...,10194.125077,-115.031001,-129.108744,74.871101,112.306652,-112.306652,0,0.000000,7960.1850,7626.8525
1804,2018-03-18 19:30:00,7606.00,7622.36,7322.00,7489.91,5606.729052,7510.0675,7674.066100,7674.066100,7322.000000,...,10188.519595,-123.670711,-128.021137,75.472944,113.209416,-113.209416,0,0.000000,7960.1850,7510.0675
1805,2018-03-18 20:30:00,7480.94,7499.50,7359.30,7385.00,2312.875660,7431.1850,7592.066800,7592.066800,7359.300000,...,10186.159962,-135.860501,-129.589010,76.265751,114.398627,-114.398627,0,0.000000,7960.1850,7431.1850
1806,2018-03-18 21:30:00,7385.00,7449.99,7330.31,7375.83,2434.776740,7385.2825,7511.625900,7511.625900,7330.310000,...,10183.654151,-147.918341,-133.254876,77.277664,115.916496,-115.916496,0,0.000000,7960.1850,7385.2825
1807,2018-03-18 22:30:00,7375.83,7511.81,7330.00,7485.00,2387.151727,7425.6600,7448.454200,7511.810000,7330.000000,...,10181.251565,-152.420795,-137.088060,78.348275,117.522412,-117.522412,0,0.000000,7960.1850,7385.2825
1808,2018-03-18 23:30:00,7489.29,7584.00,7415.53,7436.00,2365.326291,7481.2050,7437.057100,7584.000000,7415.530000,...,10178.906540,-149.635930,-139.597634,79.298851,118.948277,-118.948277,0,0.000000,7960.1850,7385.2825
1809,2018-03-19 00:30:00,7440.00,7521.00,7379.82,7463.53,1771.887512,7451.0875,7459.131050,7521.000000,7379.820000,...,10177.137113,-148.410761,-141.360259,80.143013,120.214520,-120.214520,0,0.000000,7960.1850,7385.2825
